In [1]:
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
#!pip3 install tensorflow
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from sklearn.model_selection import KFold


In [2]:
df_final_train = read_hdf('Recommend_sample_stage4.h5', 'train_df',mode='r')
df_final_train.head()

,source,destination,connected,jaccard_followers,jaccard_followees,cosine_followers,cosine_followees,num_followers_s,num_followers_d,num_followees_s,...,page_rank_s,page_rank_d,katz_s,katz_d,hubs_s,hubs_d,authorities_s,authorities_d,prefer_Attach_followers,prefer_Attach_followees
0,735020,1846324,0,0.0,0.000000,0.0,0.000000,391,10,652,...,0.000006,5.642064e-07,0.002674,0.000750,4.269629e-12,3.114232e-14,2.924335e-12,6.938837e-15,3910,5216
1,735020,1698702,0,0.0,0.000000,0.0,0.000000,391,20,652,...,0.000006,2.092265e-06,0.002674,0.000785,4.269629e-12,2.960183e-15,2.924335e-12,4.375862e-15,7820,11084
2,735020,621115,0,0.0,0.000000,0.0,0.000000,391,1,652,...,0.000006,1.520216e-07,0.002674,0.000714,4.269629e-12,3.965015e-16,2.924335e-12,1.829239e-15,391,652
3,735020,408522,0,0.0,0.002920,0.0,0.013240,391,35,652,...,0.000006,2.107550e-06,0.002674,0.000853,4.269629e-12,3.051558e-15,2.924335e-12,1.387670e-14,13685,22820
4,735020,420335,0,0.0,0.001531,0.0,0.027692,391,5,652,...,0.000006,4.014519e-07,0.002674,0.000730,4.269629e-12,2.616759e-16,2.924335e-12,3.640880e-15,1955,1304


In [3]:
y_train = df_final_train.connected

In [4]:
df_final_train.drop(['source', 'destination','connected'],axis=1,inplace=True)

## Xgboost

In [5]:
import pickle
filename = 'finalized_model_xgb.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [6]:
probs=loaded_model.predict_proba(df_final_train)

In [7]:
data_xgb=pd.DataFrame()
data_xgb['xgb']=pd.DataFrame(probs).sort_values(by=[1])[0]
data_xgb.head(20)

,xgb
17,0.933584
28,0.826614
24,0.775686
2,0.736840
47,0.496458
43,0.093270
46,0.060581
18,0.054040
12,0.041127
42,0.018388


## LGB

In [8]:
import pickle
filename = 'finalized_model_lgb.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [9]:
probs=loaded_model.predict_proba(df_final_train)

In [10]:
data_lgb=pd.DataFrame()
data_lgb['lgb']=pd.DataFrame(probs).sort_values(by=[1])[0]
data_lgb.head(20)                                    

,lgb
17,0.489008
2,0.482756
24,0.465487
28,0.465187
47,0.459313
9,0.308932
42,0.305532
15,0.304383
43,0.302258
41,0.248895


## Random Forest

In [11]:
import pickle
filename = 'finalized_model_rfc.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [12]:
probs=loaded_model.predict_proba(df_final_train)

In [13]:
data_rfc=pd.DataFrame()
data_rfc['rfc']=pd.DataFrame(probs).sort_values(by=[1])[0]
data_rfc.head(20)

,rfc
17,0.489008
2,0.482756
24,0.465487
28,0.465187
47,0.459313
9,0.308932
42,0.305532
15,0.304383
43,0.302258
41,0.248895
